In [20]:
#!pip install rasterio

# Definitions

In [17]:
MIN_PUMPKIN_WIDTH = 3
MIN_PUMPKIN_HEIGHT = 3
FILTER = "Gaussian"

WINDOW_WIDTH = 3000
WINDOW_HEIGTH = 3000
OVERLAP_WINDOW = 100

# Functions

In [22]:
import cv2
import numpy as np
from itertools import product
from rasterio import windows

BGR = "BGR"
CieLAB = "CieLAB"

def segmentation(img: np.ndarray, method: str = CieLAB) -> np.ndarray:
    """Segmentation of an image to find a color spectrum

    Args:
        img (np.ndarray): Image used for segmentation
        type (str, optional): Type of image used for segmentation. Defaults to BGR.

    Returns:
        np.ndarray: Image containing the orange color
    """

    # Fetch the lower and upper bounds
    lower_bound = np.loadtxt(f"Output/1/1/{method}/lower.txt", delimiter=' ')
    upper_bound = np.loadtxt(f"Output/1/1/{method}/upper.txt", delimiter=' ')
    ref = np.loadtxt(f"Output/1/1/{method}/mean.txt", delimiter=' ')
    # TODO
    ref = np.loadtxt(f"Output/1/1/BGR/mean.txt", delimiter=' ')

    # Copy the image
    img_tmp = img.copy()

    if method == BGR:
        lower_bound = np.loadtxt("Output/1/1/BGR/lower.txt", delimiter=' ')
        upper_bound = np.loadtxt("Output/1/1/BGR/upper.txt", delimiter=' ')
        mask = cv2.inRange(img_tmp, lower_bound, upper_bound)
        img_test = cv2.bitwise_and(img_tmp, img_tmp, mask=mask)
        return img_test
    if method == CieLAB:
        img_tmp = cv2.cvtColor(img_tmp, cv2.COLOR_BGR2LAB)
        lower_bound = np.loadtxt(f"Output/1/1/CieLAB/lower.txt", delimiter=' ')
        upper_bound = np.loadtxt(f"Output/1/1/CieLAB/upper.txt", delimiter=' ')
        mask = cv2.inRange(img_tmp, lower_bound, upper_bound)
        img_test = cv2.bitwise_and(img_tmp, img_tmp, mask=mask)


    # Distance in RGB space to a reference color
    pixels = np.reshape(img_tmp, (-1, 3))
    cov = np.cov(pixels.transpose())
    diff = pixels - np.repeat([ref], np.ma.size(pixels,axis=0),axis=0)
    # Calculate the distances
    mahalanobis = np.sqrt(np.sum(diff*diff , axis=1))  
    # Reshaping to image dimension
    mahalanobis_img = np.reshape(mahalanobis,(img_tmp.shape[0],img_tmp.shape[1]))
    # Scaling to 8 bit greyscale image
    scaled_mahalanobis_img = 255*mahalanobis_img/np.max(mahalanobis_img)
    
    # Setting up a threshhold for mahalanobis
    ret,threshold_img = cv2.threshold(scaled_mahalanobis_img,135,255, cv2.THRESH_BINARY_INV)
    return threshold_img.astype(np.uint8)



def get_tiles(ds, width=WINDOW_WIDTH, height=WINDOW_HEIGTH):
    """Creating wwindows of a given size for the orthomosaic

    Args:
        ds (DatasetReader object): Rasterio.open return value for tif file
        width (int, optional): Desired window width
        height (int, optional): Desired window height
        
    Yields:
        windows.Window: Current window for processing
    """
    
    # Getting number of coloums and rows and set offsets
    nols, nrows = ds.meta['width'], ds.meta['height']
    offsets = product(range(0, nols, width-OVERLAP_WINDOW), range(0, nrows, height-OVERLAP_WINDOW))
    big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
    for col_off, row_off in  offsets:
        window =windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, ds.transform)
        yield window, transform




def count_pumpkins_overlap(img: np.ndarray) -> int:
    """Count the number of pumpkins in the image

    Args:
        img (np.ndarray): Image used for segmentation

    Returns:
        int: Number of pumpkins
    """
    # Filter
    if "Open" == FILTER:
        kernel = np.ones((3, 3), np.uint8)
        filtered = cv2.erode(img, kernel, iterations=1)
        filtered = cv2.dilate(filtered, kernel, iterations=1)
    elif "Gaussian" == FILTER:
        filtered = cv2.GaussianBlur(img, (3,3), cv2.BORDER_DEFAULT)
    elif "Median" == FILTER:
        filtered = cv2.medianBlur(img, 3)
    else:
        filtered = img
    segmented = segmentation(filtered, method=CieLAB)

    contours, hierarchy = cv2.findContours(segmented, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    pumpkins:int = 0
    for c in contours:
        x,y,w,h = cv2.boundingRect(c)
        if w > MIN_PUMPKIN_WIDTH and h > MIN_PUMPKIN_HEIGHT:
            #if x < WINDOW_WIDTH-(OVERLAP_WINDOW/2) and y < WINDOW_HEIGTH-(OVERLAP_WINDOW/2):
            pumpkins += 1
    return pumpkins


# 3.1 Colour based segmentation
This part consists of using colour based information
to segment individual images from a pumpkin field.
The segmented images should end up having a black
background with smaller white objects on top.

## Exercise 3.1.1
Annotate some pumpkins in a test image and extract information about the average pumpkin colour
in the annotated pixels. Calculate both mean value
and standard variation. Use the following two colour
spaces: RGB and CieLAB. Finally try to visualise
the distribution of colour values.

### BGR

In [22]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img_true = cv2.imread("Output/1/1/BGR/Ground_true_EB-02-660_0594_0326.JPG")

img = cv2.imread('Images/EB-02-660_0594_0326.JPG')
cv2.imwrite("Output/1/1/BGR/img.png", img)

# lower bound and upper bound for red color
lower_bound = np.array([0, 0, 250])
upper_bound = np.array([0, 0, 255])

# Find the colors within the boundaries
mask = cv2.inRange(img_true, lower_bound, upper_bound)
cv2.imwrite("output/1/1/BGR/mask.png", mask)
mask_pixels = np.reshape(mask, (-1))

img = cv2.bitwise_and(img, img, mask=mask)
pixels = np.reshape(img, (-1, 3))
cv2.imwrite("output/1/1/BGR/RGB.png", img)

# Mean and standard deviation
mean, std = cv2.meanStdDev(img, mask=mask)
print(f"Mean color values of the annotated pixels:\n {mean}")
print(f"Standard deviation of color values of the annotated pixels:\n {std}")

# Covariance
cov = np.cov(pixels.transpose(), aweights=mask_pixels)
print(f"Covariance matrix of color values of the annotated pixels:\n {cov}")

# Save the parameters
np.savetxt("Output/1/1/BGR/lower.txt", mean-std, delimiter=' ', fmt='%1.4f')
np.savetxt("Output/1/1/BGR/upper.txt", mean+std, delimiter=' ', fmt='%1.4f')
np.savetxt("Output/1/1/BGR/mean.txt", mean, delimiter=' ', fmt='%1.4f')


# Test the output
img = cv2.imread('Images/EB-02-660_0594_0344.JPG')
mask = cv2.inRange(img, mean-std, mean+std)
img_test = cv2.bitwise_and(img, img, mask=mask)
cv2.imwrite("Output/1/1/BGR/test.png", img_test)

# Show the histogram
colors = ['b', 'g', 'r']
for dim in range(img_test.ndim):
    data = np.reshape(img_test[:, :, dim], (-1))
    plt.hist(data[data != 0], bins=255, color=colors[dim], alpha=0.5)
plt.savefig("Output/1/1/BGR/histogram.png")
plt.close()


Mean color values of the annotated pixels:
 [[ 46.41416428]
 [150.11533625]
 [230.19802827]]
Standard deviation of color values of the annotated pixels:
 [[21.71128969]
 [16.11388805]
 [17.11028936]]
Covariance matrix of color values of the annotated pixels:
 [[471.40381144 205.71747089 131.0308569 ]
 [205.71747089 259.6704494  231.02876867]
 [131.0308569  231.02876867 292.77672851]]


### CieLAB

In [23]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img_true = cv2.imread('Output/1/1/CieLAB/Ground_true_EB-02-660_0594_0326.JPG')
img = cv2.imread('Images/EB-02-660_0594_0326.JPG')

# lower bound and upper bound for red color
lower_bound = np.array([0, 0, 250])
upper_bound = np.array([0, 0, 255])

# Find the colors within the boundaries
mask = cv2.inRange(img_true, lower_bound, upper_bound)
cv2.imwrite("Output/1/1/CieLAB/mask.png", mask)
mask_pixels = np.reshape(mask, (-1))

img = cv2.bitwise_and(img, img, mask=mask)
img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
pixels = np.reshape(img, (-1, 3))
cv2.imwrite("Output/1/1/CieLAB/CieLAB.png", img)

# Mean and standard deviation
mean, std = cv2.meanStdDev(img, mask=mask)
print(f"Mean color values of the annotated pixels:\n {mean}")
print(f"Standard deviation of color values of the annotated pixels:\n {std}")

# Covariance
cov = np.cov(pixels.transpose(), aweights=mask_pixels)
print(f"Covariance matrix of color values of the annotated pixels:\n {cov}")

# Save the parameters
np.savetxt("Output/1/1/CieLAB/lower.txt", mean-std, delimiter=' ', fmt='%1.4f')
np.savetxt("Output/1/1/CieLAB/upper.txt", mean+std, delimiter=' ', fmt='%1.4f')
np.savetxt("Output/1/1/CieLAB/mean.txt", mean, delimiter=' ', fmt='%1.4f')

# Test the output
img = cv2.imread('Images/EB-02-660_0594_0344.JPG')
img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
mask = cv2.inRange(img, mean-std, mean+std)
img_test = cv2.bitwise_and(img, img, mask=mask)
cv2.imwrite("Output/1/1/CieLAB/test.png", img_test)

# Show the histogram
colors = ['b', 'g', 'r']
for dim in range(img_test.ndim):
    data = np.reshape(img_test[:, :, dim], (-1))
    plt.hist(data[data != 0], bins=255, color=colors[dim], alpha=0.5)
plt.savefig("Output/1/1/CieLAB/histogram.png")
plt.close()


Mean color values of the annotated pixels:
 [[175.31995372]
 [150.28308435]
 [190.01353051]]
Standard deviation of color values of the annotated pixels:
 [[14.39882563]
 [ 4.32889583]
 [ 7.2355212 ]]
Covariance matrix of color values of the annotated pixels:
 [[207.33660837 -19.72096074  25.87952376]
 [-19.72096074  18.74028175   6.27398643]
 [ 25.87952376   6.27398643  52.35540042]]


## Exercise 3.1.2
Segment the orange pumpkins from the background
using color information. Experiment with the following segmentation methods
1. inRange with RGB values
2. inRange with CieLAB values
3. Distance in RGB space to a reference colour
4. (HLS)

In [24]:
import cv2
import numpy as np

# Test RGB
img = cv2.imread('Images/EB-02-660_0594_0344.JPG')
lower_bound = np.loadtxt("Output/1/1/BGR/lower.txt", delimiter=' ')
upper_bound = np.loadtxt("Output/1/1/BGR/upper.txt", delimiter=' ')
mask = cv2.inRange(img, lower_bound, upper_bound)
img_test = cv2.bitwise_and(img, img, mask=mask)
cv2.imwrite("Output/1/2/BGR.png", img_test)

# Test the output
img = cv2.imread('Images/EB-02-660_0594_0344.JPG')
img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
lower_bound = np.loadtxt(f"Output/1/1/CieLAB/lower.txt", delimiter=' ')
upper_bound = np.loadtxt(f"Output/1/1/CieLAB/upper.txt", delimiter=' ')
mask = cv2.inRange(img, lower_bound, upper_bound)
img_test = cv2.bitwise_and(img, img, mask=mask)
cv2.imwrite("Output/1/2/CieLAB.png", img_test)


# Distance in RGB space to a reference color
ref = np.loadtxt(f"Output/1/1/BGR/mean.txt", delimiter=' ')
pixels = np.reshape(img, (-1, 3))
cov = np.cov(pixels.transpose())
diff = pixels - np.repeat([ref], np.ma.size(pixels,axis=0),axis=0)
# Calculate the distances
mahalanobis = np.sqrt(np.sum(np.dot(diff,np.linalg.inv(cov)) *diff , axis=1))  
euclidian = np.sqrt(np.sum(diff * diff, axis=1))
# Reshaping to image dimension
mahalanobis_img = np.reshape(mahalanobis,(img.shape[0],img.shape[1]))
euclidian_img = np.reshape(euclidian,(img.shape[0],img.shape[1]))
# Scaling to 8 bit greyscale image
scaled_mahalanobis_img = 255*mahalanobis_img/np.max(mahalanobis_img)
scaled_euclidian_img = 255*euclidian_img/np.max(euclidian_img)
# Saving scaled images
cv2.imwrite("Output/1/2/euclidian_dist_map.png", scaled_euclidian_img)
cv2.imwrite("Output/1/2/mahalanobis_dist_map.png", scaled_mahalanobis_img)

# Setting up a threshhold for mahalanobis
ret,threshold_img = cv2.threshold(scaled_mahalanobis_img,135,255, cv2.THRESH_BINARY_INV)
threshold_img = threshold_img.astype(np.uint8)
cv2.imwrite("Output/1/2/mahalanobis_segmented.png", threshold_img)

True

## Exercise 3.1.3
Choose one segmentation method to use for the rest
of the mini-project.

In [5]:
import cv2
import numpy as np

BGR = "BGR"
CieLAB = "CieLAB"

def segmentation(img: np.ndarray, method: str = CieLAB) -> np.ndarray:
    """Segmentation of an image to find a color spectrum

    Args:
        img (np.ndarray): Image used for segmentation
        type (str, optional): Type of image used for segmentation. Defaults to BGR.

    Returns:
        np.ndarray: Image containing the orange color
    """

    # Fetch the lower and upper bounds
    lower_bound = np.loadtxt(f"Output/1/1/{method}/lower.txt", delimiter=' ')
    upper_bound = np.loadtxt(f"Output/1/1/{method}/upper.txt", delimiter=' ')
    ref = np.loadtxt(f"Output/1/1/{method}/mean.txt", delimiter=' ')
    # TODO
    ref = np.loadtxt(f"Output/1/1/BGR/mean.txt", delimiter=' ')

    # Copy the image
    img_tmp = img.copy()

    if method == BGR:
        lower_bound = np.loadtxt("Output/1/1/BGR/lower.txt", delimiter=' ')
        upper_bound = np.loadtxt("Output/1/1/BGR/upper.txt", delimiter=' ')
        mask = cv2.inRange(img_tmp, lower_bound, upper_bound)
        img_test = cv2.bitwise_and(img_tmp, img_tmp, mask=mask)
        return img_test
    if method == CieLAB:
        img_tmp = cv2.cvtColor(img_tmp, cv2.COLOR_BGR2LAB)
        lower_bound = np.loadtxt(f"Output/1/1/CieLAB/lower.txt", delimiter=' ')
        upper_bound = np.loadtxt(f"Output/1/1/CieLAB/upper.txt", delimiter=' ')
        mask = cv2.inRange(img_tmp, lower_bound, upper_bound)
        img_test = cv2.bitwise_and(img_tmp, img_tmp, mask=mask)


    # Distance in RGB space to a reference color
    pixels = np.reshape(img_tmp, (-1, 3))
    cov = np.cov(pixels.transpose())
    diff = pixels - np.repeat([ref], np.ma.size(pixels,axis=0),axis=0)
    # Calculate the distances
    mahalanobis = np.sqrt(np.sum(np.dot(diff,np.linalg.inv(cov)) *diff , axis=1))  
    # Reshaping to image dimension
    mahalanobis_img = np.reshape(mahalanobis,(img_tmp.shape[0],img_tmp.shape[1]))
    # Scaling to 8 bit greyscale image
    scaled_mahalanobis_img = 255*mahalanobis_img/np.max(mahalanobis_img)
    
    # Setting up a threshhold for mahalanobis
    ret,threshold_img = cv2.threshold(scaled_mahalanobis_img,135,255, cv2.THRESH_BINARY_INV)
    return threshold_img.astype(np.uint8)

# Test the segmentation
img = cv2.imread('Images/EB-02-660_0594_0344.JPG')
img_test = segmentation(img, method=BGR)
cv2.imwrite("Output/1/3/BGR.png", img_test)
img_test = segmentation(img, method= CieLAB)
cv2.imwrite("Output/1/3/CieLAB.png", img_test)


True

# 3.2 Counting objects
This part is about counting objects in segmented
images and then to generate some visual output that
will help you to debug the programs.

## Exercise 3.2.1
Count the number of orange blobs in the segmented
image.

In [26]:
import cv2
import matplotlib.pyplot as plt

img = cv2.imread('Images/EB-02-660_0594_0344.JPG')
segmented = segmentation(img, method=CieLAB)
contours, hierarchy = cv2.findContours(segmented, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

# Get the width, height and position of each contour
width = []
for c in contours:
    x,y,w,h = cv2.boundingRect(c)
    if w > MIN_PUMPKIN_WIDTH and h > MIN_PUMPKIN_HEIGHT:
        width.append(w)
        cv2.circle(img, (x+int(w/2), y+int(h/2)), 10, (0, 0, 255), 2)

# Print histogram of size
plt.figure()
plt.hist(width,bins=255)
plt.savefig("Output/2/1/histogram.png")
plt.close()


print(f"Number contours: {len(contours)}")
cv2.imwrite("Output/2/1/circle_pumpkins.png", img)

Number contours: 5378


True

## Exercise 3.2.2
Filter the segmented image to remove noise.

In [27]:
import cv2
img = cv2.imread('Images/EB-02-660_0594_0344.JPG')
segmented = segmentation(img, method=CieLAB)
# Opening
kernel = np.ones((3, 3), np.uint8)
filtered = cv2.erode(segmented, kernel, iterations=1)
filtered = cv2.dilate(filtered, kernel, iterations=1)
cv2.imwrite("Output/2/2/filtered.png", filtered)


True

## Exercise 3.2.3
Count the number of orange blobs in the filtered image.

In [28]:
import cv2
import matplotlib.pyplot as plt

img = cv2.imread('Images/EB-02-660_0594_0344.JPG')
filtered = cv2.imread('Output/2/2/filtered.png')
filtered = cv2.cvtColor(filtered, cv2.COLOR_BGR2GRAY)
contours, hierarchy = cv2.findContours(filtered, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

# Get the width, height and position of each contour
width = []
for c in contours:
    x,y,w,h = cv2.boundingRect(c)
    if w > MIN_PUMPKIN_WIDTH and h > MIN_PUMPKIN_HEIGHT:
        width.append(w)
        cv2.circle(img, (x+int(w/2), y+int(h/2)), 10, (0, 0, 255), 2)

# Print histogram of size
plt.figure()
plt.hist(width,bins=255)
plt.savefig("Output/2/3/histogram.png")
plt.close()

print(f"Number contours: {len(contours)}")
cv2.imwrite("Output/2/3/circle_pumpkins.png", img)

Number contours: 3332


True

## Exercise 3.2.4
Mark the located pumpkins in the input image. This
step is for debugging purposes and to convince others
that you have counted the pumpkins accurately.

In [29]:
# See the outputs

# 3.3 Generate an orthomosaic
This part deals with orthorectifying multiple images
of the same field into a single carthometric product.
Choose proper settings for all below processes, taking into consideration the available computing resources

## Exercise 3.3.1
Load data into Metashape.

## Exercise 3.3.2
Perform bundle adjustment (align photos) and check
results

## Exercise 3.3.3
Perform dense reconstruction

## Exercise 3.3.4
Create digital elevation model

## Exercise 3.3.5
Create orthomosaic

## Exercise 3.3.6
Limit orthomosaic to pumpkin field

# 3.4 Count in orthomosaic
Use the python package rasterio to perform operations on the orthomosaic using a tile based approach.

## Exercise 3.4.1
Create code that only loads parts of the orthomosaic.

In [4]:
from itertools import product
from rasterio import windows

def get_tiles(ds, width=WINDOW_WIDTH, height=WINDOW_HEIGTH):
    """Creating wwindows of a given size for the orthomosaic

    Args:
        ds (DatasetReader object): Rasterio.open return value for tif file
        width (int, optional): Desired window width
        height (int, optional): Desired window height
        
    Yields:
        windows.Window: Current window for processing
    """
    
    # Getting number of coloums and rows and set offsets
    nols, nrows = ds.meta['width'], ds.meta['height']
    offsets = product(range(0, nols, width-OVERLAP_WINDOW), range(0, nrows, height-OVERLAP_WINDOW))
    big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
    for col_off, row_off in  offsets:
        window =windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, ds.transform)
        yield window, transform




NameError: name 'WINDOW_WIDTH' is not defined

## Exercise 3.4.2
Design tile placement incl. overlaps.

## Exercise 3.4.3
Count pumpkins in each tile.

In [3]:
import cv2
import numpy as np

def count_pumpkins(img: np.ndarray) -> int:
    """Count the number of pumpkins in the image

    Args:
        img (np.ndarray): Image used for segmentation

    Returns:
        int: Number of pumpkins
    """
    # Filter
    if "Open" == FILTER:
        kernel = np.ones((3, 3), np.uint8)
        filtered = cv2.erode(img, kernel, iterations=1)
        filtered = cv2.dilate(filtered, kernel, iterations=1)
    elif "Gaussian" == FILTER:
        filtered = cv2.GaussianBlur(img, (3,3), cv2.BORDER_DEFAULT)
    elif "Median" == FILTER:
        filtered = cv2.medianBlur(img, 3)
    else:
        filtered = img
    segmented = segmentation(filtered, method=CieLAB)

    contours, hierarchy = cv2.findContours(segmented, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    pumpkins:int = 0
    for c in contours:
        _,_,w,h = cv2.boundingRect(c)
        if w > MIN_PUMPKIN_WIDTH and h > MIN_PUMPKIN_HEIGHT:
            pumpkins += 1
    return pumpkins

img = cv2.imread('Images/EB-02-660_0594_0344.JPG')
pumpkins = count_pumpkins(img)
print(f"Number contours: {pumpkins}")

NameError: name 'FILTER' is not defined

## Exercise 3.4.4
Deal with pumpkins in the overlap, so they are only
counted once.

In [2]:
import cv2
import numpy as np

def count_pumpkins_overlap(img: np.ndarray) -> int:
    """Count the number of pumpkins in the image

    Args:
        img (np.ndarray): Image used for segmentation

    Returns:
        int: Number of pumpkins
    """
    # Filter
    if "Open" == FILTER:
        kernel = np.ones((3, 3), np.uint8)
        filtered = cv2.erode(img, kernel, iterations=1)
        filtered = cv2.dilate(filtered, kernel, iterations=1)
    elif "Gaussian" == FILTER:
        filtered = cv2.GaussianBlur(img, (3,3), cv2.BORDER_DEFAULT)
    elif "Median" == FILTER:
        filtered = cv2.medianBlur(img, 3)
    else:
        filtered = img
    segmented = segmentation(filtered, method=CieLAB)

    contours, hierarchy = cv2.findContours(segmented, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    pumpkins:int = 0
    for c in contours:
        x,y,w,h = cv2.boundingRect(c)
        if w > MIN_PUMPKIN_WIDTH and h > MIN_PUMPKIN_HEIGHT:
            if x < WINDOW_WIDTH-(OVERLAP_WINDOW/2) and y < WINDOW_HEIGTH-(OVERLAP_WINDOW/2):
                pumpkins += 1
    return pumpkins


## Exercise 3.4.5
Determine amount of pumpkins in the entire field

In [21]:
import rasterio
import cv2
from rasterio.windows import Window
from itertools import product
from rasterio import windows

filename = "./Orthomosaic.tif"

with rasterio.open(filename) as src:

    count = 0

    for window, transform in get_tiles(src):

        img = src.read(window=window)
        # Loaded image has a different shape
        # than opencv image, so...
        temp = img.transpose(1, 2, 0)
        t2 = cv2.split(temp)
        img_cv = cv2.merge([t2[2], t2[1], t2[0]])

        count += count_pumpkins_overlap(img_cv)

        print(count)
        if count ==5623:
                cv2.namedWindow("Resized_Window", cv2.WINDOW_NORMAL)
                cv2.resizeWindow("Resized_Window", 300, 700)
                cv2.imshow('Resized_Window', img_cv)
                cv2.waitKey(0)
                cv2.destroyAllWindows()
        


[[2700.66869629  127.69443831 -429.19702376]
 [ 127.69443831   38.3543187   -31.73406296]
 [-429.19702376  -31.73406296  108.84516627]]
1317
[[2122.06247716  156.46030421 -266.4636598 ]
 [ 156.46030421   68.37073222  -16.91558245]
 [-266.4636598   -16.91558245  102.03309175]]
2665
[[2928.84526607  132.10865114 -412.05002365]
 [ 132.10865114   41.53705054  -32.74189064]
 [-412.05002365  -32.74189064  103.38738487]]
4133
[[1439.77078752   87.41398626 -141.61082886]
 [  87.41398626   53.26390906  -19.54559263]
 [-141.61082886  -19.54559263   73.207033  ]]
4873
[[2993.42329757  193.83099231 -454.03657069]
 [ 193.83099231   60.90889472  -51.72880506]
 [-454.03657069  -51.72880506  118.66232225]]
6213
[[1055.9629482    96.90631425 -122.52789339]
 [  96.90631425   83.39906719  -56.18833667]
 [-122.52789339  -56.18833667   92.80206014]]
6518
[[2791.83898555  116.78820356 -375.92690694]
 [ 116.78820356   42.58853255  -26.93300905]
 [-375.92690694  -26.93300905   97.04794722]]
8117
[[2022.208529

# 3.5 Endnotes
Reflect on the conducted work in this miniproject.

## Exercise 3.5.1
Determine GSD and size of the image field. What is
the average number of pumpkins per area?

## Exercise 3.5.2
Reflect on whether the developed system is ready to
help a farmer with the task of estimating the number
of pumpkins in a field.